<a href="https://colab.research.google.com/github/mhoangvslev/RSFB/blob/main/Realistic_Synthetic_Federated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1jA4vXeLe5SDEzlxjR4T4YQgGbzX34kaN -O metrics.csv

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd


df = pd.read_csv("metrics.csv")
df

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 


g = sns.lineplot(data=df, x="batch", y="relevant_sources_selectivity", marker="o", style="query", hue="query")
g.set_xticks(df["batch"].unique())
plt.legend(labels=df["query"].sort_values().unique(), loc = 2, bbox_to_anchor = (1,1))
plt.show()